In [1]:
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const

In [2]:
# given constants 

extinction = 0.09 
F_H_beta = 10**(-10.08 + extinction) *u.erg / (u.cm)**2 / u.s

V_0 = 12.9 - 2.175 * extinction
F_ref_5480 = 3.65e-9 * 10**(-0.4*V_0) * u.erg / (u.cm)**2 / u.s / u.AA

T_neb = 9600 * u.K 

n_e = 5.1e3 * (u.cm)**(-3)


# Uncertanity

For the uncertanity of the calculated values the min-max method is used to obtain the maximal error. There is only a uncertanity known for two of the given values: F_H_beta and F_ref_5480

In [65]:
extinction_max = 0.09 + 0.07
extinction_min = 0.09 - 0.07

F_H_beta_max = 10**(-10.08 + 0.3 + extinction_max) *u.erg / (u.cm)**2 / u.s
F_H_beta_min = 10**(-10.08 - 0.3 + extinction_min) *u.erg / (u.cm)**2 / u.s

V_0_max = 12.9 + 0.3 - 2.175 * extinction_min
V_0_min = 12.9 - 0.3 - 2.175 * extinction_max

F_ref_5480_max = 3.65e-9 * 10**(-0.4*V_0_min) * u.erg / (u.cm)**2 / u.s / u.AA
F_ref_5480_min = 3.65e-9 * 10**(-0.4*V_0_max) * u.erg / (u.cm)**2 / u.s / u.AA

delta_F_H_beta = (F_H_beta_max - F_H_beta_min) / 2 
delta_F_ref_5480 = (F_ref_5480_max - F_ref_5480_min) / 2 


In [66]:
# Measurements 

# relative fluxes in equivalent-width

f_H_beta = 94.4978    # äquivalentbreite
f_He_II = 2.219203    # äquivalentbreite

# diameter of the nebula in pixels

fwhm_H = 14    # in pixels
base_H = 24    # in pixels

fwhm_He = 7    # in pixels
base_He = 15    # in pixels

fwhm_seeing = 6    # in pixels
base_seeing = 9    # in pixels

average_ratio = (fwhm_H / base_H + fwhm_He/base_He + fwhm_seeing/base_seeing) /3

ang_H = np.sqrt(fwhm_H **2 - fwhm_seeing **2) / average_ratio
ang_He = np.sqrt(fwhm_He **2 - fwhm_seeing **2) / average_ratio 

# log(g) and T_eff of the star trough model fit to H_gamma line

log_g_1 = 3.74
log_g_2 = 3.5
T_eff_1 = 5e4 * u.K
T_eff_2= 4e4 * u.K


In [67]:
# Recombination Coefficient defined

def alpha(T, z):
    '''
    Z is the nuclear charge
    C is some constant
    x is the temperature dependent variable
    '''
    C = 5.197e-14 * u.cm**3 / u.s
    x = 1.57890e5 * z**2 / (T.si.value)

    alpha = C * z * np.sqrt(x) * (0.5 * np.log(x) + 0.469 * x**(-1/3) - 0.3412)
    
    return alpha

In [68]:
# Zanstra ratios defined

def Zanstra(nu, A, Flux, Flux_ref, T, z, B_l_T):
    '''
    nu is the frequency of the line
    A is the Einstein-Coefficient of the line
    Flux is the Flux of the Line
    Flux_ref is the used reference flux
    '''
    h = 6.626196e-27 * u.erg * u.s

    Z = (h * nu * B_l_T * A / ((T) **(3/2) * alpha(T, z))
         * Flux_ref / Flux)
    
    return Z

In [69]:
# theoretical/model Zanstra Ratio

def Zanstra_model(T, wave,  freq):
    '''
    T is the blackbody temperature
    wave is the reference wavelength
    freq is the minimum ionization frrequency
    '''
    Z = planck_wave(wave, T) / ionizing_photons(freq, T)

    return Z

In [70]:
# Here the angular diameter is defined

def convert_pixel_to_angular(pixel):

    #theta is the angular diameter in arcsec
    theta = 1 / 1.82 * pixel * u.arcsec

    return theta

In [71]:
# Planck function in wavelength config defined

def planck_wave(wave, T):
    h = const.h
    kB = const.k_B
    c = const.c

    B = 2 * h * c**2 / wave**5 / (np.exp(h * c / (wave * kB * T)) - 1)
    
    return B.decompose().value * 1e-7 * u.erg / u.s / u.cm **2 / u.AA

In [72]:
# number of ionizing photons in the wiens approximation

def ionizing_photons(nu, T):
    '''
    nu frequency where the ionisation becomes possible
    T is the blackbody temperature
    '''
    h = const.h
    kB = const.k_B
    c = const.c

    N = (2 / c**2 * np.exp(-h * nu / (kB * T)) * 
         (nu **2 * kB * T / h 
          + 2 * nu * (kB * T / h)**2 
          + 2 * (kB * T / h)**3))
    
    return N.si.value * 1e-4 / u.s / u.cm**2

In [73]:
# 1. Angular diameters 

ang_dia_H = convert_pixel_to_angular(ang_H)
ang_dia_He = convert_pixel_to_angular(ang_He)

print(f"ang_dia_H = {ang_dia_H}")
print("---------------")
print(f"ang_dia_He = {ang_dia_He}")

# full real angular diameter of the nebula ngc 6210 is 40"x30", BUT
# the luminous center is 13"x16" (according to wikipedia)
# He is more centered, H value corresponds roughly!

ang_dia_H = 12.145737305671783 arcsec
---------------
ang_dia_He = 3.462067798909196 arcsec


In [86]:
# 2. Flux of He and alpha He

alpha_H_beta = F_H_beta / f_H_beta
alpha_He_II = (alpha_H_beta
               * convert_pixel_to_angular(ang_He)**2
               / convert_pixel_to_angular(ang_H)**2)
F_He_II = alpha_He_II * f_He_II * 10**(1.04 * extinction)

# error

alpha_H_max = (F_H_beta + delta_F_H_beta) / f_H_beta
alpha_H_min = (F_H_beta - delta_F_H_beta) / f_H_beta

alpha_He_max = (alpha_H_max
               * convert_pixel_to_angular(ang_He)**2
               / convert_pixel_to_angular(ang_H)**2)
alpha_He_min = (alpha_H_min
               * convert_pixel_to_angular(ang_He)**2
               / convert_pixel_to_angular(ang_H)**2)
F_He_max = alpha_He_max * f_He_II * 10**(1.04 * extinction_max)
F_He_min = alpha_He_min * f_He_II * 10**(1.04 * extinction_min)

print(F_He_min)

delta_F_He = (F_He_max - F_He_min) / 2

print(f"F_He_II = {F_He_II} +- {delta_F_He}")


8.434047070082103e-15 erg / (s cm2)
F_He_II = 2.4221369929782024e-13 erg / (s cm2) +- 2.763032586955446e-13 erg / (s cm2)


In [75]:
F_H_beta


<Quantity 1.02329299e-10 erg / (s cm2)>

In [76]:
# 3. Zanstra ratios 

# H_beta

A_H_beta = 8.42e6 / u.s
nu_H_beta = (4861.3 * u.AA).to(u.Hz, equivalencies=u.spectral()).decompose()
B_H_beta = 3.58e-15 * u.K **(3/2) * u.cm **3
z_H = 1 

Z_H = Zanstra(nu_H_beta, A_H_beta, F_H_beta, F_ref_5480,
                            T_neb, z_H, B_H_beta)


# He 

A_He = 142.4e6 / u.s
nu_He = (4686.0 * u.AA).to(u.Hz, equivalencies=u.spectral()).decompose()
B_He = 1.44e-15 * u.K **(3/2) * u.cm **3
z_He = 2

Z_He = Zanstra(nu_He, A_He, F_He_II, F_ref_5480, T_neb, z_He, B_He)


# error

Z_H_max = Zanstra(nu_H_beta, A_H_beta, F_H_beta_min, F_ref_5480_max, 
                  T_neb, z_H, B_H_beta)
Z_H_min = Zanstra(nu_H_beta, A_H_beta, F_H_beta_max, F_ref_5480_min, 
                  T_neb, z_H, B_H_beta)

delta_Z_H = (Z_H_max - Z_H_min) / 2

Z_He_max = Zanstra(nu_He, A_He, F_He_min, F_ref_5480_max, 
                  T_neb, z_He, B_He)
Z_He_min = Zanstra(nu_He, A_He, F_He_max, F_ref_5480_min, 
                  T_neb, z_He, B_He)

delta_Z_He = (Z_He_max - Z_He_min) / 2

print(f"Z_H_beta = {Z_H} +- {delta_Z_H}")
print("-------------------------")
print(f"Z_He = {Z_He} +- {delta_Z_He}")


Z_H_beta = 1.4768863529479585e-16 erg / Angstrom +- 2.417839869849598e-16 erg / Angstrom
-------------------------
Z_He = 7.325989493766626e-14 erg / Angstrom +- 1.5850814177036363e-12 erg / Angstrom


In [77]:
# 4. theoretical Zanstra Ratio 

# blackbody for many temperatures

T_eff_array = np.array([40,45, 50, 55, 60, 65, 70, 75, 80]) * 1e3 * u.K

print(f"Flux at 5480 Angström: {planck_wave(5480 * u.AA, T_eff_array)}")

print("-------------------")
# ionizing photons
#nu_min = wave_ref.to(u.Hz, equivalencies=u.spectral()).decompose()

#print(ionizing_photons(nu_min, T_eff_array))

# model Zanstra ratios 

wave_ref = (5480 * u.AA)
nu_min_H = (13.6 * 1.60218e-19 * u.J / const.h ).decompose()
nu_min_He = (4 * 13.6 * 1.60218e-19 * u.J / const.h ).decompose()

for i in range(len(T_eff_array)):
    T_eff = T_eff_array[i]
    Z_H = Zanstra_model(T_eff, wave_ref, nu_min_H)  # Compute for H
    print(f"Zanstra model ratios H: T = {T_eff:.0f}  K, Z = {Z_H:.3e}")
   
print("-------------------")

for i in range(len(T_eff_array)):
    T_eff = T_eff_array[i]
    Z_H = Zanstra_model(T_eff, wave_ref, nu_min_He)  # Compute for H
    print(f"Zanstra model ratios H: T = {T_eff:.0f}  K, Z = {Z_H:.3e}")


Flux at 5480 Angström: [2.59760497e+08 3.04220261e+08 3.48963016e+08 3.93912955e+08
 4.39019046e+08 4.84245698e+08 5.29567351e+08 5.74965178e+08
 6.20425010e+08] erg / (Angstrom s cm2)
-------------------
Zanstra model ratios H: T = 40000 K  K, Z = 4.095e-16 erg / Angstrom
Zanstra model ratios H: T = 45000 K  K, Z = 2.595e-16 erg / Angstrom
Zanstra model ratios H: T = 50000 K  K, Z = 1.782e-16 erg / Angstrom
Zanstra model ratios H: T = 55000 K  K, Z = 1.298e-16 erg / Angstrom
Zanstra model ratios H: T = 60000 K  K, Z = 9.882e-17 erg / Angstrom
Zanstra model ratios H: T = 65000 K  K, Z = 7.787e-17 erg / Angstrom
Zanstra model ratios H: T = 70000 K  K, Z = 6.306e-17 erg / Angstrom
Zanstra model ratios H: T = 75000 K  K, Z = 5.220e-17 erg / Angstrom
Zanstra model ratios H: T = 80000 K  K, Z = 4.400e-17 erg / Angstrom
-------------------
Zanstra model ratios H: T = 40000 K  K, Z = 5.098e-12 erg / Angstrom
Zanstra model ratios H: T = 45000 K  K, Z = 9.046e-13 erg / Angstrom
Zanstra model ra

In [15]:
T_eff_H = 55e3 * u.K
T_eff_He = 55e3 * u.K

In [42]:
# 5. T_eff and log_g using Table 3

table_3_H = np.log10(1 / Z_H.value)
table_3_He = np.log10(1 / Z_He.value)

delta_table_3_H = (np.log10(1 / Z_H_min.value) - np.log10(1 / Z_H_max.value)) /2
delta_table_3_He = (np.log10(1 / Z_He_min.value) - np.log10(1 / Z_He_max.value)) /2


print(f"table 3 H = {table_3_H} +- {delta_table_3_H}")
print(f"table 3 He = {table_3_He} +- {delta_table_3_He}")


# Hydrogen

log_NH_F = 15.83
T_eff_H_table_3 = 50000 * u.K
log_g_H_table_3 = 4.5

log_NHe_F = 13.13
T_eff_He_table_3 = 80000 * u.K
log_g_He_table_3 = 5.5

table 3 H = 15.830652922553 +- 0.5509000000000013
table 3 He = 13.135133708494632 +- 1.092379121183698


In [17]:
# 6. log_g and T_eff with the fit of model spectra

print(f"log_g_1 = , {log_g_1}, T_eff = {50e3 * u.K}")
print(f"log_g_2 = , {log_g_2}, T_eff = {40e3 * u.K}")



log_g_1 = , 3.74, T_eff = 50000.0 K
log_g_2 = , 3.5, T_eff = 40000.0 K


In [81]:
# 7./8. R_CSPN and M_CSPN
M_cspn = 0.552 * const.M_sun
#print(M_cspn)

print("-----------------")

g = 10**(4.5) * u.cm / u.s**2
R_star_squared = (const.G * M_cspn / g).decompose()
R_star = np.sqrt(R_star_squared)

print(R_star / const.R_sun)

-----------------
0.6918363114967312


In [ ]:
# 9. CSPN distance 
distance = R_star_squared * planck_wave(5480*u.AA, 55e3 * u.K) / F_ref_5480
d = (np.sqrt(distance.decompose())/ (1*u.pc)).decompose().value * u.pc

delta_distance = ((np.sqrt((R_star_squared * planck_wave(5480*u.AA, T_eff_H_table_3) / F_ref_5480_min).decompose())
                  - np.sqrt((R_star_squared * planck_wave(5480*u.AA, T_eff_H_table_3) / F_ref_5480_max)).decompose()) / (2 * u.pc)).decompose() * u.pc

print(f"distance = {d} +- {delta_distance}")



distance = 1780.2398182840748 pc +- 351.50172803985777 pc
348963015.66399646 erg / (Angstrom s cm2)


In [85]:
# 10. radius, mass and kinematic age of the nebula

nu = nu_H_beta
n_4 = n_e **2 * 3.58e-15 * u.K **(3/2) * u.cm **3 / T_neb **(3/2)
A_42 = 8.42e6 / u.s
pc = 3086e16 * u.m    # distance conversion pc in m
ly = 9.461e15 * u.m    # distance conversion light years in m

r_neb = 4.848 * 1e-6 * ang_dia_H.value / 2 * d 

delta_r_neb = (((4.848 * 1e-6 * ang_dia_H.value / 2 * (d+delta_distance))
                 -( 4.848 * 1e-6 * ang_dia_H.value / 2 * (d-delta_distance))) / 2)

rho_mean = (n_e / 1.13 * (1 + 4 * 0.1) * 1.00784 * u.u).decompose()


M_neb = (4 * np.pi * d**2 * F_H_beta / (const.h * nu * n_4 * A_42)
          * rho_mean / const.M_sun)

delta_M_neb = (((4 * np.pi * d**2 * F_H_beta_max / (const.h * nu * n_4 * A_42)
          * rho_mean / const.M_sun) - 
          (4 * np.pi * d**2 * F_H_beta_min / (const.h * nu * n_4 * A_42)
          * rho_mean / const.M_sun)) / 2)



print(f"R_neb = {(r_neb/ly).decompose()} +- {(delta_r_neb/ly).decompose()} Ly")

print("--------------")

print(f"M_neb = {M_neb.decompose()} +- {delta_M_neb.decompose()} M_sun")


print("-----------------")
# Kinematic age

t_kin = r_neb.decompose() / (21 * u.km / u.s)

delta_t_kin = delta_r_neb.decompose() / (21 * u.km / u.s)

print((f"t_kin = {t_kin.decompose().value/(365 * 24 * 3600)}" 
      + f" +- {delta_t_kin.decompose().value/(365 * 24 * 3600)} years"))

R_neb = 0.17094189440323607 +- 0.033751840993569086 Ly
--------------
M_neb = 0.06058562493343817 +- 0.05809099051636571 M_sun
-----------------
t_kin = 2442.078687016828 +- 482.17935004010104 years
